## Import libraries

In [1]:
import numpy as np
from os import path, listdir

from brainpipe.system import study
from brainpipe.feature import TF
from brainpipe.visual import *

In [8]:
sf = 512
lines = [0]
f = (1,29,1,1)  # Frequency vector: (from, to, width, step) 1 to 120 Hz 4 2 #0.1, 13, 0.5, 0.1
baseline = [1280,1408] # 250ms Time in seconds (rest 700ms au milieu des 1s extraite) Odor (1408,1536) Rest 640,768
width, step = 80, 20 #Time vector
win_sel = [0,2] # In seconds (start RSA analyses from 0s)

### Compute TF by odor single-trial
250ms of baseline (-500 to -250ms before inspiration onset)

In [10]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/Encoding_By_Odor/')
path2save = path.join (st.path, 'feature/1_TF_E_single_trial_rsa_by_odor/')
subjects = ['LEFC','CHAF','VACJ','SEMC','FERJ','PIRJ']
rois_selected = ['OFC','HC','IFG']

files = listdir(path_data)

for file in files:
    splits = file.split('_')
    if splits[0] in subjects and splits[-1] == 'physFT.npz':
        mat = np.load(path_data+file,allow_pickle=True)
        print('processing --> ',file)
        labels, data = mat['Mai_RL'], mat['x'] #data (n elecs x n pts x ntrials)
        channels, xyz = mat['channels'], mat['xyz']
        
        for roi in rois_selected:
            print('processing ROI ',roi)
            savename = path2save+file.replace('bipo_sel_physFT.npz', roi+'_theta_TF.npz')
            idx_rois = np.where([1 if lab == roi else 0 for lab in labels])[0]
            data_sel = data[idx_rois,:,:]
            chan_sel, xyz_sel = channels[idx_rois], xyz[idx_rois]
            n_elecs, npts, ntrials = data_sel.shape

            if n_elecs > 0 and not path.exists(savename):
                kwargs = {} # Define an empty dictionnary to save all power parameters
                kwargs['f'] = f
                kwargs['time_width'], kwargs['time_step'] = width, step # take power in 358 samples (700ms) windows width every 51 samples (100ms)
                kwargs['channels'], kwargs['xyz_sel'] = chan_sel, xyz_sel
                time = (np.arange(npts)-1536)/sf

                tfObj = TF(sf, npts, f=f, time=time, width=width, step=step,
                           baseline=baseline,norm=3)

                TF_by_trial = np.array([])
                for trial in range(ntrials):
                    x = data_sel[:,:,trial][:,:,np.newaxis]
                    print('x shape', x.shape)
                    x_tf = tfObj.get(x)[0][:,:,:,np.newaxis]
                    print('TF one trial shape',x_tf.shape)
                    TF_by_trial = np.concatenate((TF_by_trial,x_tf),axis=-1) if np.size(TF_by_trial) else x_tf
                print('TF_all_trials shape',TF_by_trial.shape)
                
                #select only relevant timepoints
                timebin = np.array(tfObj.xvec)
                sl = slice(np.argmin(np.abs(timebin-win_sel[0])), np.argmin(np.abs(timebin-win_sel[1])))
                kwargs['TF'] = TF_by_trial[:,:,sl,:]
                kwargs['time'] = timebin[sl]
                
                print('check selection',kwargs['TF'].shape,kwargs['time'].shape)
#                 print('check selection',kwargs['time'])
                np.savez(savename, **kwargs)
                del kwargs['TF']
                del kwargs, x, n_elecs, ntrials, npts
            del data_sel

-> Olfacto loaded
processing -->  CHAF_odor_1_bipo_sel_physFT.npz
processing ROI  OFC
processing ROI  HC
processing ROI  IFG
processing -->  CHAF_odor_2_bipo_sel_physFT.npz
processing ROI  OFC
processing ROI  HC
processing ROI  IFG
processing -->  CHAF_odor_3_bipo_sel_physFT.npz
processing ROI  OFC
processing ROI  HC
processing ROI  IFG
processing -->  LEFC_odor_14_bipo_sel_physFT.npz
processing ROI  OFC
x shape (7, 3584, 1)
TF one trial shape (7, 7, 176, 1)
x shape (7, 3584, 1)
TF one trial shape (7, 7, 176, 1)
x shape (7, 3584, 1)
TF one trial shape (7, 7, 176, 1)
x shape (7, 3584, 1)
TF one trial shape (7, 7, 176, 1)
x shape (7, 3584, 1)
TF one trial shape (7, 7, 176, 1)
x shape (7, 3584, 1)
TF one trial shape (7, 7, 176, 1)
TF_all_trials shape (7, 7, 176, 6)
check selection (7, 7, 51, 6) (51,)
processing ROI  HC
x shape (8, 3584, 1)
TF one trial shape (7, 8, 176, 1)
x shape (8, 3584, 1)
TF one trial shape (7, 8, 176, 1)
x shape (8, 3584, 1)
TF one trial shape (7, 8, 176, 1)
x shape

TF one trial shape (7, 9, 176, 1)
x shape (9, 3584, 1)
TF one trial shape (7, 9, 176, 1)
x shape (9, 3584, 1)
TF one trial shape (7, 9, 176, 1)
x shape (9, 3584, 1)
TF one trial shape (7, 9, 176, 1)
x shape (9, 3584, 1)
TF one trial shape (7, 9, 176, 1)
x shape (9, 3584, 1)
TF one trial shape (7, 9, 176, 1)
TF_all_trials shape (7, 9, 176, 6)
check selection (7, 9, 51, 6) (51,)
processing ROI  HC
x shape (13, 3584, 1)
TF one trial shape (7, 13, 176, 1)
x shape (13, 3584, 1)
TF one trial shape (7, 13, 176, 1)
x shape (13, 3584, 1)
TF one trial shape (7, 13, 176, 1)
x shape (13, 3584, 1)
TF one trial shape (7, 13, 176, 1)
x shape (13, 3584, 1)
TF one trial shape (7, 13, 176, 1)
x shape (13, 3584, 1)
TF one trial shape (7, 13, 176, 1)
TF_all_trials shape (7, 13, 176, 6)
check selection (7, 13, 51, 6) (51,)
processing ROI  IFG
x shape (2, 3584, 1)
TF one trial shape (7, 2, 176, 1)
x shape (2, 3584, 1)
TF one trial shape (7, 2, 176, 1)
x shape (2, 3584, 1)
TF one trial shape (7, 2, 176, 1)
x

TF one trial shape (7, 14, 176, 1)
x shape (14, 3584, 1)
TF one trial shape (7, 14, 176, 1)
x shape (14, 3584, 1)
TF one trial shape (7, 14, 176, 1)
TF_all_trials shape (7, 14, 176, 3)
check selection (7, 14, 51, 3) (51,)
processing -->  VACJ_odor_10_bipo_sel_physFT.npz
processing ROI  OFC
x shape (15, 3584, 1)
TF one trial shape (7, 15, 176, 1)
x shape (15, 3584, 1)
TF one trial shape (7, 15, 176, 1)
x shape (15, 3584, 1)
TF one trial shape (7, 15, 176, 1)
TF_all_trials shape (7, 15, 176, 3)
check selection (7, 15, 51, 3) (51,)
processing ROI  HC
x shape (7, 3584, 1)
TF one trial shape (7, 7, 176, 1)
x shape (7, 3584, 1)
TF one trial shape (7, 7, 176, 1)
x shape (7, 3584, 1)
TF one trial shape (7, 7, 176, 1)
TF_all_trials shape (7, 7, 176, 3)
check selection (7, 7, 51, 3) (51,)
processing ROI  IFG
x shape (10, 3584, 1)
TF one trial shape (7, 10, 176, 1)
x shape (10, 3584, 1)
TF one trial shape (7, 10, 176, 1)
x shape (10, 3584, 1)
TF one trial shape (7, 10, 176, 1)
TF_all_trials shape

TF one trial shape (7, 4, 176, 1)
x shape (4, 3584, 1)
TF one trial shape (7, 4, 176, 1)
x shape (4, 3584, 1)
TF one trial shape (7, 4, 176, 1)
x shape (4, 3584, 1)
TF one trial shape (7, 4, 176, 1)
x shape (4, 3584, 1)
TF one trial shape (7, 4, 176, 1)
x shape (4, 3584, 1)
TF one trial shape (7, 4, 176, 1)
x shape (4, 3584, 1)
TF one trial shape (7, 4, 176, 1)
x shape (4, 3584, 1)
TF one trial shape (7, 4, 176, 1)
TF_all_trials shape (7, 4, 176, 10)
check selection (7, 4, 51, 10) (51,)
processing ROI  IFG
x shape (14, 3584, 1)
TF one trial shape (7, 14, 176, 1)
x shape (14, 3584, 1)
TF one trial shape (7, 14, 176, 1)
x shape (14, 3584, 1)
TF one trial shape (7, 14, 176, 1)
x shape (14, 3584, 1)
TF one trial shape (7, 14, 176, 1)
x shape (14, 3584, 1)
TF one trial shape (7, 14, 176, 1)
x shape (14, 3584, 1)
TF one trial shape (7, 14, 176, 1)
x shape (14, 3584, 1)
TF one trial shape (7, 14, 176, 1)
x shape (14, 3584, 1)
TF one trial shape (7, 14, 176, 1)
x shape (14, 3584, 1)
TF one tri

TF one trial shape (7, 6, 176, 1)
x shape (6, 3584, 1)
TF one trial shape (7, 6, 176, 1)
TF_all_trials shape (7, 6, 176, 3)
check selection (7, 6, 51, 3) (51,)
processing ROI  HC
x shape (12, 3584, 1)
TF one trial shape (7, 12, 176, 1)
x shape (12, 3584, 1)
TF one trial shape (7, 12, 176, 1)
x shape (12, 3584, 1)
TF one trial shape (7, 12, 176, 1)
TF_all_trials shape (7, 12, 176, 3)
check selection (7, 12, 51, 3) (51,)
processing ROI  IFG
x shape (4, 3584, 1)
TF one trial shape (7, 4, 176, 1)
x shape (4, 3584, 1)
TF one trial shape (7, 4, 176, 1)
x shape (4, 3584, 1)
TF one trial shape (7, 4, 176, 1)
TF_all_trials shape (7, 4, 176, 3)
check selection (7, 4, 51, 3) (51,)
processing -->  VACJ_odor_13_bipo_sel_physFT.npz
processing ROI  OFC
x shape (15, 3584, 1)
TF one trial shape (7, 15, 176, 1)
x shape (15, 3584, 1)
TF one trial shape (7, 15, 176, 1)
x shape (15, 3584, 1)
TF one trial shape (7, 15, 176, 1)
x shape (15, 3584, 1)
TF one trial shape (7, 15, 176, 1)
TF_all_trials shape (7, 

TF one trial shape (7, 7, 176, 1)
x shape (7, 3584, 1)
TF one trial shape (7, 7, 176, 1)
x shape (7, 3584, 1)
TF one trial shape (7, 7, 176, 1)
TF_all_trials shape (7, 7, 176, 4)
check selection (7, 7, 51, 4) (51,)
processing ROI  HC
x shape (8, 3584, 1)
TF one trial shape (7, 8, 176, 1)
x shape (8, 3584, 1)
TF one trial shape (7, 8, 176, 1)
x shape (8, 3584, 1)
TF one trial shape (7, 8, 176, 1)
x shape (8, 3584, 1)
TF one trial shape (7, 8, 176, 1)
TF_all_trials shape (7, 8, 176, 4)
check selection (7, 8, 51, 4) (51,)
processing ROI  IFG
x shape (3, 3584, 1)
TF one trial shape (7, 3, 176, 1)
x shape (3, 3584, 1)
TF one trial shape (7, 3, 176, 1)
x shape (3, 3584, 1)
TF one trial shape (7, 3, 176, 1)
x shape (3, 3584, 1)
TF one trial shape (7, 3, 176, 1)
TF_all_trials shape (7, 3, 176, 4)
check selection (7, 3, 51, 4) (51,)
